In [2]:
from IPython.display import display, Image
from openai import OpenAI
from PIL import Image
import json
import os
import io
import pandas as pd
import requests



In [1]:
client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY",""))


KeyboardInterrupt



In [38]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user, user_message,file_ids=None):
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }

    if file_ids:
        params['file_ids'] = file_ids
    
    client.beta.threads.messages.create(
        **params
    )

    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)
    

In [39]:
financial_data_path = 'data/NotRealCorp_financial_data.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head()

,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


In [40]:
file = client.files.create(
    file=open('data/NotRealCorp_financial_data.json','rb'),
    purpose='assistants'
)

In [41]:
file1 = client.files.create(
    file=open('data/Steps to develop POC using Open AI.docx','rb'),
    purpose='assistants'
)

In [42]:
assistant = client.beta.assistants.create(
    name='NewDocuAssistant',
    instructions='Sonata GenAI POC',
    model='gpt-3.5-turbo-1106',
    tools=[{"type":"code_interpreter"}],
    file_ids=[file.id,file1.id]
)

In [43]:
thread = client.beta.threads.create(
    messages=[
        {
            'role':'user',
            'file_ids':[file.id],
            'content':'Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue'
        }
    ]
)

In [44]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [45]:
messages = client.beta.threads.messages.list(
    thread_id= thread.id
)
show_json(messages)

{'data': [{'id': 'msg_PQnpjQTyzcjf8xHFud4UCX1j',
   'assistant_id': 'asst_jtPoYWdy1se3445cdJlABAbH',
   'content': [{'text': {'annotations': [],
      'value': 'First, I will examine the contents of the uploaded file to understand its structure and extract the necessary information to calculate the profit by quarter and year.'},
     'type': 'text'}],
   'created_at': 1709011232,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_IFaBl5Axj8KLJnxNl0X54YOs',
   'thread_id': 'thread_rkCtyIq8iwDHdsmn6FDe48s1'},
  {'id': 'msg_KFVfhVi7nKPsJh9C6uZ6HCpx',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue'},
     'type': 'text'}],
   'created_at': 1709011227,
   'file_ids': ['file-dYK59OJ9L279qTjfYP75lFiR'],
   'met

In [46]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        messages.data[0].content[0].image_file
        time.sleep(5)
        print('Plot created')
        break
    except:
        time.sleep(10)
        print("Assistant is still working.....")

Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistant is still working.....
Assistan

KeyboardInterrupt: 

In [ ]:
messages = client.beta.threads.runs.list(
    thread_id=thread.id,
)
[message.content[0] for message in messages.data]

In [ ]:
def convert_file_to_png(file_id,write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path,'wb') as file:
        file.write(data_bytes)
        

In [ ]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = '../images/NotRealCorp_chart.png'
convert_file_to_png(plot_file_id, image_path)

plot_file = client.files.create(
    file=open(image_path, 'rb'),
    purpose='assistants'
)